In [31]:
#!pip install xgboost
#!pip install scikit-multilearn

In [250]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
# from better_profanity import profanity
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import Ridge #import ridge 

#imports
%matplotlib inline
import numpy as np 
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest

from skmultilearn.problem_transform import BinaryRelevance

In [ ]:
#pop rock folk
#balancing response for multilabel data creates information loss.  here is how we dealt with this.

In [310]:
#import rock balanced data

X_train_balance = pd.read_pickle('veclyrics_balanced_rock_doc2vec_train.pkl')
X_test_balance = pd.read_pickle('veclyrics_balanced_rock_doc2vec_test.pkl')

y_train_balance = pd.read_pickle('response_balanced_rock_doc2vec_train.pkl')
y_train_balance = np.array(y_train_balance.iloc[:,3:10])

y_test_balance = pd.read_pickle('response_balanced_rock_doc2vec_test.pkl')
y_test_balance = np.array(y_test_balance.iloc[:,3:10])

y_train_bal1 = y_train_balance[:,0]
y_test_bal1 = y_test_balance[:,0]
y_train_bal2 = y_train_balance[:,1]
y_test_bal2 = y_test_balance[:,1]
y_train_bal3 = y_train_balance[:,2]
y_test_bal3 = y_test_balance[:,2]
y_train_bal4 = y_train_balance[:,3]
y_test_bal4 = y_test_balance[:,3]
y_train_bal5 = y_train_balance[:,4]
y_test_bal5 = y_test_balance[:,4]
y_train_bal6 = y_train_balance[:,5]
y_test_bal6 = y_test_balance[:,5]
y_train_bal7 = y_train_balance[:,6]
y_test_bal7 = y_test_balance[:,6]

In [313]:
def eval_balance(pred):
    
    #pred (function input) is array of predictions made using X test
    #y_test_vec (imported elsewhere) is array of y test
    
    acc = []

    for i in range(0,7):
        check = np.array(pred[:,i]) == y_test_balance[:,i]
        accuracy = check.sum()/len(check)
        acc.append(accuracy)
    
    null = pd.DataFrame(pd.DataFrame(y_test_balance).sum()/len(pd.DataFrame(y_test_balance)))
    null.iloc[1:,0] = 1-null.iloc[1:,0]
    
    comp = pd.DataFrame({'accuracy':list(acc), 'null':list(null[0])})
    comp['diff'] = comp['accuracy'] - comp['null']
    
    return(comp)

In [315]:
#initialize and fit models
#doc2vec rock balanced
# ~5 min runtime

gb_bal1 = GradientBoostingClassifier()
gb_bal1.fit(X_train_balance,y_train_bal1)

gb_bal2 = GradientBoostingClassifier()
gb_bal2.fit(X_train_balance,y_train_bal2)

gb_bal3 = GradientBoostingClassifier()
gb_bal3.fit(X_train_balance,y_train_bal3)

gb_bal4 = GradientBoostingClassifier()
gb_bal4.fit(X_train_balance,y_train_bal4)

gb_bal5 = GradientBoostingClassifier()
gb_bal5.fit(X_train_balance,y_train_bal5)

gb_bal6 = GradientBoostingClassifier()
gb_bal6.fit(X_train_balance,y_train_bal6)

gb_bal7 = GradientBoostingClassifier()
gb_bal7.fit(X_train_balance,y_train_bal7)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [317]:
#test and evaluate models
gb_bal1pred = gb_bal1.predict(X_test_balance)
gb_bal2pred = gb_bal2.predict(X_test_balance)
gb_bal3pred = gb_bal3.predict(X_test_balance)
gb_bal4pred = gb_bal4.predict(X_test_balance)
gb_bal5pred = gb_bal5.predict(X_test_balance)
gb_bal6pred = gb_bal6.predict(X_test_balance)
gb_bal7pred = gb_bal7.predict(X_test_balance)

gb_pred_bal = pd.DataFrame({'rock':gb_bal1pred, 'singer-songwriter':gb_bal2pred, 'pop':gb_bal3pred, 
              'metal':gb_bal4pred, 'folk':gb_bal5pred, 'country':gb_bal6pred, 'hip hop / rap':gb_bal7pred})

gb_pred_bal = np.array(gb_pred_bal)

eval_balance(gb_pred_bal)

,accuracy,null,diff
0,0.569961,0.501934,0.068027
1,0.868347,0.868747,-0.000400
2,0.599040,0.574763,0.024276
3,0.887822,0.888089,-0.000267
4,0.848473,0.848739,-0.000267
5,0.853808,0.853808,0.000000
6,0.923836,0.903962,0.019875


In [334]:
non = gb_bal1.predict(X_test_balance)

In [335]:
grid = gsgbc.predict(X_test_balance)

In [336]:
gb_bal1.score(X_test_balance,y_test_bal1)

0.5699613178604775

In [337]:
gsgbc.score(X_test_balance,y_test_bal1)

0.5716953448045885

In [338]:
gsgbc.score(X_train_balance,y_train_bal1)

0.7371264674493063

In [342]:
gsgbc.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=5,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=3, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=3, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [343]:
# grid search Gradient boosting classifier for balanced rock column only
gb = GradientBoostingClassifier(n_estimators=1000, random_state=3)
# params_gbc = {
#     "learning_rate": [0.01, 0.05, 0.10, 0.15],
#     "max_depth": [3, 4, 5, 6],
#     "min_samples_leaf": [1, 2, 3, 4, 5],
#     "max_features":['sqrt']
# }

params_gbc = {
    "learning_rate": [0.01, 0.05, 0.10],
    "max_depth": [4, 5, 6],
    "min_samples_leaf": [2, 3, 4],
    "max_features":['sqrt']
}

#reducing from 10 with kfold to cv = 5 to halve fits
gbtune = GridSearchCV(gb, param_grid=params_gbc, cv=10, scoring="accuracy", n_jobs= -1,verbose = 10)

gbtune.fit(X_train_balance,y_train_bal1)

Fitting 10 folds for each of 27 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 24.8min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 28.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 35.7min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 40.2min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 47.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 52

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=1000,
                                                  n_iter_no_change=None,
                                                  presor

In [344]:
# Best score
print(gbtune.best_score_)

#test score
print(gbtune.score(X_test_balance,y_test_bal1))

#train score
print(gbtune.score(X_train_balance,y_train_bal1))

0.6567169156883671
0.578764839269041
0.7879869263607258


In [348]:
# save the model
filename = 'gbtune.sav'
pickle.dump(gbtune, open(filename, 'wb'))

In [349]:
# load the model
filename = 'gbtune.sav'
best = pickle.load(open(filename, 'rb'))

In [350]:
best.score(X_test_balance,y_test_bal1)

0.578764839269041

In [323]:
# grid search Gradient boosting classifier
gb = GradientBoostingClassifier(n_estimators=200, random_state=3)
# params_gbc = {
#     "learning_rate": [0.01, 0.05, 0.10, 0.15],
#     "max_depth": [3, 4, 5, 6],
#     "min_samples_leaf": [1, 2, 3, 4, 5],
#     "max_features":['sqrt']
# }

params_gbc = {
    "learning_rate": [0.01, 0.05, 0.10],
    "max_depth": [3, 4, 5],
    "min_samples_leaf": [1, 2, 3],
    "max_features":['sqrt']
}

#reducing from 10 with kfold to cv = 5 to halve fits
gsgbc = GridSearchCV(gb, param_grid=params_gbc, cv=3, scoring="accuracy", n_jobs= -1,verbose = 10)

#gb_best = gsgbc.best_estimator_

gsgbc.fit(X_train_balance,y_train_bal1)

# Best score
gsgbc.best_score_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   40.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  75 out of  81 | elapsed:  3.1min remaining:   15.0s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  3.3min finished


0.6500133404482391

In [341]:
gsgbc.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=5,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=3, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=3, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [333]:
gsgbc.best_score_

0.6500133404482391

In [326]:
gsgbc.score(X_test_balance,y_test_bal1)

0.5716953448045885

In [347]:
com = y_test_bal1 == rockpred

array([False, False,  True, ..., False,  True,  True])

In [332]:
rockparam = gsgbc.best_estimator_
rockpred = rockparam.predict(X_test_balance)
rockparam.score(X_test_balance,y_test_bal1)

0.5716953448045885

In [25]:
#how i would normally check an individual response class
y_pred_rf = rockparam.predict(X_test)
y_pred_train_rf = rockparam.predict(X_train)
print('test report',classification_report(y_test_one, y_pred_rf))
print('train report',classification_report(y_train_one, y_pred_train_rf))

test report               precision    recall  f1-score   support

           0       0.60      0.13      0.21      2459
           1       0.78      0.97      0.87      7766

    accuracy                           0.77     10225
   macro avg       0.69      0.55      0.54     10225
weighted avg       0.74      0.77      0.71     10225

train report               precision    recall  f1-score   support

           0       0.92      0.36      0.52     10190
           1       0.82      0.99      0.90     30708

    accuracy                           0.83     40898
   macro avg       0.87      0.67      0.71     40898
weighted avg       0.85      0.83      0.80     40898



In [18]:
gsgbc.cv_results_

{'mean_fit_time': array([11.3081518 , 13.08590802, 13.12594668, 18.31042401, 19.43094103,
        20.38704999, 27.49654754, 27.80691083, 28.40115023, 15.38440625,
        13.60730775, 11.39607056, 17.16316581, 19.0936594 , 19.81371172,
        26.59359026, 28.10804582, 28.44664598, 16.39201482, 12.78171674,
        10.58074252, 15.82891202, 17.96121391, 19.3528076 , 27.18937818,
        25.85446143, 19.18196519]),
 'std_fit_time': array([1.26626   , 0.27484841, 0.23021418, 0.63289025, 0.45235387,
        0.38011948, 0.87657483, 0.86250159, 1.3267893 , 0.42033844,
        0.75131222, 0.21753072, 0.23068524, 0.82817441, 0.25773188,
        0.39976925, 1.24255933, 2.07529208, 0.48978439, 0.97744221,
        0.21217617, 0.91924774, 0.60534562, 0.33500888, 0.83075881,
        1.36594184, 3.06516744]),
 'mean_score_time': array([0.13264505, 0.12400134, 0.12765892, 0.18417533, 0.15658085,
        0.16954732, 0.21841685, 0.20811049, 0.1863389 , 0.14062421,
        0.12765892, 0.11968017, 0.160

In [321]:
#balance
#fitting random forest classifier
#works for multilabel response data by default
rf_balance = RandomForestClassifier(verbose = True, n_jobs= -1)
rf_balance.fit(X_train_balance, y_train_balance)
rf_pred_balance = rf_balance.predict(X_test_balance)

C:\Users\peter\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.8s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


In [356]:
# save random forest model
filename = 'rfmulti.sav'
pickle.dump(rf_balance, open(filename, 'wb'))

In [ ]:
#how to load and run default random forest (poor performance as example) and tuned gradient boosting (good performance)

#import rock balanced data
X_train_balance = pd.read_pickle('veclyrics_balanced_rock_doc2vec_train.pkl')
X_test_balance = pd.read_pickle('veclyrics_balanced_rock_doc2vec_test.pkl')

y_train_balance = pd.read_pickle('response_balanced_rock_doc2vec_train.pkl')
y_train_balance = np.array(y_train_balance.iloc[:,3:10])

y_test_balance = pd.read_pickle('response_balanced_rock_doc2vec_test.pkl')
y_test_balance = np.array(y_test_balance.iloc[:,3:10])

# load tuned gradient boosting single column (balanced rock) model
filename = 'gbtune.sav'
gbtune = pickle.load(open(filename, 'rb'))

# load random forest model for multi-label where only rock is balanced
filename = 'rfmulti.sav'
rf_balance = pickle.load(open(filename, 'rb'))

#predict with rf
rf_pred_balance = rf_balance.predict(X_test_balance)

#predict with gb
gb_pred_balance = gbtune.predict(X_test_balance)

In [371]:
breaX = pd.read_pickle('veclyrics_doc2vec_holdout.pkl')
holdoutresponse = gbtune.predict(breaX)
pickle.dump(holdoutresponse, open("holdoutresponse.pkl", 'wb'))

In [365]:
len(X_test_balance)

7497

In [358]:
gbtune.predict(X_test_balance)

array([1, 0, 1, ..., 0, 1, 1])

In [351]:
eval_balance(rf_pred_balance)

,accuracy,null,diff
0,0.542750,0.501934,0.040816
1,0.866213,0.868747,-0.002534
2,0.579432,0.574763,0.004669
3,0.887955,0.888089,-0.000133
4,0.845271,0.848739,-0.003468
5,0.852074,0.853808,-0.001734
6,0.909564,0.903962,0.005602


In [318]:
#initialize and fit models
#rock balanced
#few second runtime

lr_bal1 = LogisticRegression(solver='liblinear')
lr_bal1.fit(X_train_balance,y_train_bal1)

lr_bal2 = LogisticRegression(solver='liblinear')
lr_bal2.fit(X_train_balance,y_train_bal2)

lr_bal3 = LogisticRegression(solver='liblinear')
lr_bal3.fit(X_train_balance,y_train_bal3)

lr_bal4 = LogisticRegression(solver='liblinear')
lr_bal4.fit(X_train_balance,y_train_bal4)

lr_bal5 = LogisticRegression(solver='liblinear')
lr_bal5.fit(X_train_balance,y_train_bal5)

lr_bal6 = LogisticRegression(solver='liblinear')
lr_bal6.fit(X_train_balance,y_train_bal6)

lr_bal7 = LogisticRegression(solver='liblinear')
lr_bal7.fit(X_train_balance,y_train_bal7)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [320]:
#test and evaluate models
lr_bal1pred = lr_bal1.predict(X_test_balance)
lr_bal2pred = lr_bal2.predict(X_test_balance)
lr_bal3pred = lr_bal3.predict(X_test_balance)
lr_bal4pred = lr_bal4.predict(X_test_balance)
lr_bal5pred = lr_bal5.predict(X_test_balance)
lr_bal6pred = lr_bal6.predict(X_test_balance)
lr_bal7pred = lr_bal7.predict(X_test_balance)

lr_pred_bal = pd.DataFrame({'rock':lr_bal1pred, 'singer-songwriter':lr_bal2pred, 'pop':lr_bal3pred, 
              'metal':lr_bal4pred, 'folk':lr_bal5pred, 'country':lr_bal6pred, 'hip hop / rap':lr_bal7pred})

lr_pred_bal = np.array(lr_pred_bal)

eval_balance(lr_pred_bal)

,accuracy,null,diff
0,0.579032,0.501934,0.077098
1,0.865946,0.868747,-0.002801
2,0.604642,0.574763,0.029879
3,0.887822,0.888089,-0.000267
4,0.844605,0.848739,-0.004135
5,0.853675,0.853808,-0.000133
6,0.927037,0.903962,0.023076


In [229]:
#test and evaluate models
lr1pred = lr1.predict(X_test_vec)
lr2pred = lr2.predict(X_test_vec)
lr3pred = lr3.predict(X_test_vec)
lr4pred = lr4.predict(X_test_vec)
lr5pred = lr5.predict(X_test_vec)
lr6pred = lr6.predict(X_test_vec)
lr7pred = lr7.predict(X_test_vec)

lr_pred_vec = pd.DataFrame({'rock':lr1pred, 'singer-songwriter':lr2pred, 'pop':lr3pred, 
              'metal':lr4pred, 'folk':lr5pred, 'country':lr6pred, 'hip hop / rap':lr7pred})

lr_pred_vec = np.array(lr_pred_vec)

eval(lr_pred_vec)

,accuracy,null,diff
0,0.764010,0.759511,0.004499
1,0.805477,0.821711,-0.016235
2,0.570367,0.553545,0.016822
3,0.851247,0.850562,0.000685
4,0.771051,0.792665,-0.021614
5,0.807335,0.816919,-0.009584
6,0.933985,0.927335,0.006650
